In [18]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf
import matplotlib.pyplot as plt
import cv2
import PIL
import pathlib

from PIL import Image
from sklearn.model_selection import train_test_split  
from tensorflow import keras as tfkeras
from tensorflow.keras import layers, models, callbacks, preprocessing, applications
from tensorflow.keras.preprocessing.image import img_to_array, load_img, ImageDataGenerator
from tensorflow.keras.layers.experimental import preprocessing
from tensorflow.keras.applications import VGG16

In [19]:
data_path = '/kaggle/input/rice-image-dataset/Rice_Image_Dataset'
data_dir = pathlib.Path(data_path)

width=224
height=224

print(data_dir)

/kaggle/input/rice-image-dataset/Rice_Image_Dataset


In [20]:
batch_size = 32

train_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(height, width),
  batch_size=batch_size
)

val_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(height, width),
  batch_size=batch_size
) 

# print(train_ds.shape)

Found 75000 files belonging to 5 classes.
Using 60000 files for training.
Found 75000 files belonging to 5 classes.
Using 15000 files for validation.


In [21]:
# pretrained_vgg16 = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# for layer in pretrained_vgg16.layers:
#     layer.trainable = False

In [32]:
channels = 3

image_model = models.Sequential([
    preprocessing.RandomFlip('horizontal'), # flip left-to-right
    preprocessing.RandomContrast(0.5),
    pretrained_vgg16,
    layers.Conv2D(filters=32, kernel_size=5, activation="relu", padding='same'),
                  # give the input dimensions in the first layer
                  # [height, width, color channels(RGB)]
#     layers.MaxPool2D(),
    layers.Conv2D(filters=64, kernel_size=3, activation="relu", padding='same'),
#     layers.MaxPool2D(),
    layers.Conv2D(filters=128, kernel_size=3, activation="relu", padding='same'),
    layers.MaxPool2D(),
    layers.Dropout(0.2),
    # Classifier Head
    layers.Flatten(),
    layers.Dense(units=6, activation="relu"),
    layers.Dense(5, activation='softmax')
]) 

# image_model.summary()

In [33]:
image_model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

In [ ]:
# Training
earlystop = tfkeras.callbacks.EarlyStopping(
    patience=5,
    verbose=1
)

history = image_model.fit(
    train_ds, 
    validation_data=val_ds,
    epochs=20,
    callbacks=[earlystop]
)

Epoch 1/20
1875/1875 [==============================] - 185s 99ms/step - loss: 0.0715 - accuracy: 0.9809 - val_loss: 0.0601 - val_accuracy: 0.9860
Epoch 2/20
1875/1875 [==============================] - 184s 98ms/step - loss: 0.0557 - accuracy: 0.9847 - val_loss: 0.0522 - val_accuracy: 0.9887
Epoch 3/20
1875/1875 [==============================] - 184s 98ms/step - loss: 0.0532 - accuracy: 0.9849 - val_loss: 0.0527 - val_accuracy: 0.9871
Epoch 4/20
1875/1875 [==============================] - 185s 98ms/step - loss: 0.0531 - accuracy: 0.9852 - val_loss: 0.0523 - val_accuracy: 0.9867
Epoch 5/20
1875/1875 [==============================] - 185s 98ms/step - loss: 0.0463 - accuracy: 0.9867 - val_loss: 0.0806 - val_accuracy: 0.9809
Epoch 6/20
  52/1875 [..............................] - ETA: 2:29 - loss: 0.0465 - accuracy: 0.9868

In [ ]:
test_acc = model.evaluate(val_ds)

print(f'Loss: {test_acc[0]}')
print(f'Accuracy: {100*(test_acc[1])}%')

In [ ]:
classes = {
    0: 'Arborio rice',
    1: 'Basmati rice'
}
img_path = "/kaggle/input/testing-images/julie.jpg"
fileImage = Image.open(img_path).convert("RGB").resize([width,height],Image.LANCZOS)
image = np.array(fileImage)
myimage = image.reshape(1, width,height,3)

# prepare pixel data
myimage = myimage.astype('float32')
myimage = myimage/255.

plt.figure(figsize = (4,2))
plt.imshow(image)